# Variable Selection

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import keras
import warnings

from sklearn.preprocessing import StandardScaler, OneHotEncoder, label_binarize, MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from keras.models import Sequential
from keras.layers import Dense
from hpelm import ELM
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, auc, accuracy_score
warnings.filterwarnings("ignore")

2023-05-02 03:38:22.523682: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data = pd.read_csv("Group_14_data_cleaned.csv", index_col=0)

In [20]:
# Split the data into training and testing sets
X = data.drop(['Class', 'Bidder_ID'], axis=1)
y = data['Class']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler


# Standardize the features using the StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Fit a Lasso regression model on the training data
alpha = 0.004  # regularization strength
lasso = Lasso(alpha=alpha)
lasso.fit(X_train, y_train)

# Select the features with non-zero coefficients
selected_features_Lasso = X.columns[lasso.coef_ != 0]

# Print the selected features
print("Selected features using Lasso:", selected_features_Lasso)


corr = data.corr()['Class'].sort_values(ascending=False)
selected_features = corr[corr > 0.1].index.tolist()
selected_features.remove('Class')

# Print the selected features
print("Selected features using correlations: ", selected_features)


Selected features using Lasso: Index(['Successive_Outbidding', 'Last_Bidding', 'Winning_Ratio',
       'Auction_Duration'],
      dtype='object')
Selected features using correlations:  ['Successive_Outbidding', 'Bidding_Ratio', 'Winning_Ratio', 'Bidder_Tendency']


In [6]:
print(corr)

Class                     1.000000
Successive_Outbidding     0.901035
Bidding_Ratio             0.569435
Winning_Ratio             0.394122
Bidder_Tendency           0.295533
Last_Bidding              0.097655
Early_Bidding             0.053570
Auction_Bids              0.044964
Starting_Price_Average    0.042604
Auction_Duration          0.021145
Name: Class, dtype: float64


let's look at correlations. Correlation measures how strong a relationship exists between two variables. In feature selection, we look at the correlation between the independent variables and the target variable. If the correlation is high, it indicates that the independent variable has a strong relationship with the target variable and hence, may be a good predictor. In this case, we can see that 'Successive_Outbidding', 'Starting_Price_Average', 'Early_Bidding', 'Winning_Ratio', and 'Class' are highly correlated with each other.

However, correlation alone may not be the best method for feature selection, as it only measures linear relationships between variables. It may not capture the complex relationships that may exist in the data. In other words, two variables may be highly related, but not necessarily in a linear fashion.

This is where Lasso comes in. Lasso is a regularization technique that is commonly used for feature selection. It works by shrinking the coefficients of less important variables towards zero, effectively reducing the number of features in the model. Lasso is particularly useful when dealing with high-dimensional data, where the number of features is much larger than the number of samples.

In this case, we can see that Lasso has selected a subset of features that are important for predicting the target variable, namely 'Bidder_Tendency', 'Successive_Outbidding', 'Last_Bidding', 'Starting_Price_Average', 'Winning_Ratio', and 'Auction_Duration'. By using Lasso, we can effectively reduce the number of features while still maintaining good predictive performance.

# Training the best models from part 2 and part 3  on selected features using **LASSO**


In [29]:
# Split the data into training and testing sets
X = data[['Bidder_Tendency', 'Successive_Outbidding', 'Last_Bidding',
       'Winning_Ratio']]
# X = data[['Successive_Outbidding', 'Bidding_Ratio', 'Winning_Ratio', 'Bidder_Tendency']]
y = data['Class']
X_train_selected, X_test_selected, y_train_selected, y_test_selected = train_test_split(X, y, test_size=0.2)

In [30]:
# Define the neural network model
def create_model_(num_layers=1, num_neurons=10):
    model = Sequential()
    model.add(Dense(num_neurons, input_dim=X_train_selected.shape[1], activation='relu'))
    for i in range(num_layers):
        model.add(Dense(num_neurons, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [31]:
model = create_model_(num_layers=2, num_neurons=64)
model.fit(X_train_selected, y_train_selected, epochs=50, verbose=2)

Epoch 1/50
158/158 - 1s - loss: 0.2040 - accuracy: 0.9337 - 622ms/epoch - 4ms/step
Epoch 2/50
158/158 - 0s - loss: 0.0442 - accuracy: 0.9802 - 199ms/epoch - 1ms/step
Epoch 3/50
158/158 - 0s - loss: 0.0435 - accuracy: 0.9812 - 205ms/epoch - 1ms/step
Epoch 4/50
158/158 - 0s - loss: 0.0412 - accuracy: 0.9818 - 219ms/epoch - 1ms/step
Epoch 5/50
158/158 - 0s - loss: 0.0415 - accuracy: 0.9806 - 200ms/epoch - 1ms/step
Epoch 6/50
158/158 - 0s - loss: 0.0400 - accuracy: 0.9814 - 207ms/epoch - 1ms/step
Epoch 7/50
158/158 - 0s - loss: 0.0418 - accuracy: 0.9804 - 205ms/epoch - 1ms/step
Epoch 8/50
158/158 - 0s - loss: 0.0405 - accuracy: 0.9800 - 198ms/epoch - 1ms/step
Epoch 9/50
158/158 - 0s - loss: 0.0392 - accuracy: 0.9832 - 200ms/epoch - 1ms/step
Epoch 10/50
158/158 - 0s - loss: 0.0393 - accuracy: 0.9818 - 201ms/epoch - 1ms/step
Epoch 11/50
158/158 - 0s - loss: 0.0390 - accuracy: 0.9822 - 202ms/epoch - 1ms/step
Epoch 12/50
158/158 - 0s - loss: 0.0392 - accuracy: 0.9804 - 201ms/epoch - 1ms/step
E

In [32]:
# BEST MODEL FROM PART - 2 (Random Forest)
# Best hyperparameters:  
# {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}

# Define the random forest model
rf_model = RandomForestClassifier(n_estimators=100, max_depth=20, min_samples_leaf=1, min_samples_split=5)
rf_model.fit(X_train_selected, y_train_selected)
y_pred_rf = rf_model.predict(X_test_selected)
acc_rf = accuracy_score(y_test, y_pred_rf)

print(acc_rf)

0.8205533596837945


In [28]:
# Evaluate the model on the testing set
loss, accuracy_nn = model.evaluate(X_test_selected, y_test_selected)
# Print the accuracy without rounding
print("Accuracy of the best part 2 model (Rf) on selected features : {:.4f}".format(acc_rf))
print("Accuracy of the best part 3 model (NN) on selected features: {:.4f}".format(accuracy_nn))

40/40 [==============================] - 0s 1ms/step - loss: 0.0422 - accuracy: 0.9810
Accuracy of the best part 2 model (Rf) on selected features : 0.7968
Accuracy of the best part 3 model (NN) on selected features: 0.9810
